<a href="https://colab.research.google.com/github/DennisPSmith5676/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataframe Basics

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [ ]:
# Download the Postgres driver that will allow spark to interact with Postgres
!wget https://jdbc.postgressql.org/download/postgressql-42.2.16.Javascript

--2022-03-31 20:08:37--  https://jdbc.postgressql.org/download/postgressql-42.2.16.Javascript
Resolving jdbc.postgressql.org (jdbc.postgressql.org)... 104.200.22.130, 104.200.23.95
Connecting to jdbc.postgressql.org (jdbc.postgressql.org)|104.200.22.130|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘jdbc.postgressql.org’.
To connect to jdbc.postgressql.org insecurely, use `--no-check-certificate'.


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16_Amazon-Pet-Product-Review").config("spark.driver.extraClassPath", "/content/postgressql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://amazonvineanalysisdps5676.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [ ]:
# Print our schema
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



Step 1
Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.



In [11]:
# Create the vine_table. DataFrame
df_reviews = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df_reviews.printSchema()
df_reviews.show()

root
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          2|            0|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
|          2|            0|          0|   N|                Y|
|          5|            1|          1|   N|                N|
|          5|         

In [14]:
# filter reviews with >=20 votes
df_reviews_20 = df_reviews.filter(df_reviews["total_votes"] >= 20)
df_reviews_20.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



Step 2
Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [28]:
# filter df_reviews_20 for all rows where helpful_votes / Total_votes >=0.5
df_reviews_50 = df_reviews_20.filter(df_reviews_20["helpful_votes"]/df_reviews_20["total_votes"] >= 0.50)

df_reviews_50.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



Step 3
Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [41]:
# filter df_reviews_50 for all rows where vine = "Y"
paid_reviews = df_reviews_50.filter(df_reviews_50["vine"] == "Y")

paid_reviews.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|           27|         30|   Y|                N|
|          5|           72|         72|   Y|                N|
|          5|           39|         42|   Y|                N|
|          5|           29|         30|   Y|                N|
|          2|           20|         20|   Y|                N|
|          4|           25|         26|   Y|                N|
|          3|           24|         25|   Y|                N|
|          3|           55|         58|   Y|                N|
|          3|           27|         36|   Y|                N|
|          5|          216|        219|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [30]:
# explore summary stats using describe
paid_reviews.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|               170|               170|               170| 170|              170|
|   mean| 3.929411764705882|63.641176470588235| 68.03529411764706|null|             null|
| stddev|1.1124708391270801|107.53304464635706|110.81862649267784|null|             null|
|    min|                 1|               106|               100|   Y|                N|
|    max|                 5|                99|                96|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



Step 4
Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [31]:
# filter df_reviews_50 for all rows where vine = "N"
unpaid_reviews = df_reviews_50.filter(df_reviews_50["vine"] == "N")

unpaid_reviews.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [32]:
# explore summary stats using describe
unpaid_reviews.describe().show()

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             37840|             37840|             37840|37840|            37840|
|   mean|3.7647463002114163| 57.50052854122622|61.863583509513745| null|             null|
| stddev|1.6055561346315663|107.60541722886983|111.92734632290345| null|             null|
|    min|                 1|                10|               100|    N|                N|
|    max|                 5|               998|               995|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



Step 5 
Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [39]:
# determine count of paid 5 star reviews
paid_5star_reviews = paid_reviews[paid_reviews['star_rating'] == 5].count()
paid_5star_reviews

65

In [46]:
# determine count of paid reviews
paid_reviews_df = paid_reviews.count()
paid_reviews_df

170

In [48]:
# determine % of Paid 5 star reviews
percentage_paid_5star_reviews = paid_5star_reviews / paid_reviews_df * 100
percentage_paid_5star_reviews

38.23529411764706

In [50]:
# determine count of unpaid 5 star reviews
unpaid_5star_reviews = unpaid_reviews[unpaid_reviews['star_rating'] == 5].count()
unpaid_5star_reviews

20612

In [51]:
# determine count of unpaid reviews
unpaid_reviews_df = unpaid_reviews.count()
unpaid_reviews_df

37840

In [52]:
# determine % of unPaid 5 star reviews
percentage_unpaid_5star_reviews = unpaid_5star_reviews / unpaid_reviews_df * 100
percentage_unpaid_5star_reviews

54.471458773784356

Analysis

Ho: 5star_vine = 5star_not_vine (there is no difference in the % (proportion) of 5 star ratings paid (vine) vs unpaid (not vine)
Ha: 5star_vine = 5star_not_vine (there is no difference in the % (proportion) of 5 star ratings paid (vine) vs unpaid (not vine)